In [1]:
import pandas as pd
import numpy as np
import xarray as xr 
import rioxarray as rio 

import rasterio

## Xarray

Converting the NetCDF4 file that stores the information for the machine-learning Human Footprint Index (ml-HFI) results into a DataFrame for inspection and analysis.

from https://stackoverflow.com/questions/66169106/transform-part-of-a-netcdf-file-into-a-dataframe-with-xarray:

In [9]:
dp = xr.open_dataset('./data/ml_hfi_v1_2000.nc')

ml = dp.to_dataframe()
ml = ml.dropna().reset_index()
print(ml.head())

          lat        lon  __xarray_dataarray_variable__
0  -55.609663 -68.108226                   3.328762e-06
1  -55.609663 -68.098333                   1.328018e-08
2  -55.609663 -68.088440                   1.276196e-13
3  -55.609663 -68.078547                   7.129802e-14
4  -55.609663 -68.068655                   6.915253e-18
5  -55.609663 -68.058762                   8.362481e-10
6  -55.609663 -68.048869                   4.444112e-13
7  -55.609663 -68.038976                   6.182441e-06
8  -55.609663 -68.029084                   7.117076e-07
9  -55.599770 -68.226938                   3.421031e-08
10 -55.599770 -68.217046                   4.486133e-09
11 -55.599770 -68.207153                   6.264514e-09
12 -55.599770 -68.197260                   5.907112e-08
13 -55.599770 -68.187367                   1.511353e-05
14 -55.599770 -68.118118                   5.960242e-10


In [10]:
ml.tail()

,lat,lon,__xarray_dataarray_variable__
140472840,69.988495,171.830159,1.399456e-07
140472841,69.988495,171.840051,2.790310e-04
140472842,69.988495,171.849944,3.740434e-05
140472843,69.988495,171.859837,4.128010e-02
140472844,69.988495,171.869730,8.492768e-04


### Created desired subset for AOI:
(40,30N & -110,-100W) which aligns with the Hansen Global Forest Cover tile that I've downloaded.

In [2]:
dp = xr.open_dataset('./data/ml_hfi_v1_2019.nc')

In [3]:
dp

<xarray.Dataset>
Dimensions:                        (lat: 13141, lon: 36390)
Coordinates:
  * lat                            (lat) float64 -59.99 -59.98 ... 69.99 70.0
  * lon                            (lon) float64 -180.0 -180.0 ... 180.0 180.0
Data variables:
    __xarray_dataarray_variable__  (lat, lon) float64 ...

In [4]:
dp['__xarray_dataarray_variable__']

<xarray.DataArray '__xarray_dataarray_variable__' (lat: 13141, lon: 36390)>
[478200990 values with dtype=float64]
Coordinates:
  * lat      (lat) float64 -59.99 -59.98 -59.97 -59.96 ... 69.98 69.99 70.0
  * lon      (lon) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0

In [5]:
# From https://stackoverflow.com/questions/29135885/netcdf4-extract-for-subset-of-lat-lon

ds = xr.open_dataset('./data/ml_hfi_v1_2019.nc')
lat_bnds, lon_bnds = [30, 40], [-110, -100]
subset = ds.sel(lat=slice(*lat_bnds), lon=slice(*lon_bnds))
subset

<xarray.Dataset>
Dimensions:                        (lat: 1011, lon: 1011)
Coordinates:
  * lat                            (lat) float64 30.0 30.01 ... 39.98 39.99
  * lon                            (lon) float64 -110.0 -110.0 ... -100.0 -100.0
Data variables:
    __xarray_dataarray_variable__  (lat, lon) float64 ...

In [55]:
type(subset)

xarray.core.dataset.Dataset

In [56]:
# Saving dataarray to netCDF
subset.to_netcdf('./data/ml_hfi_subset.nc')

### Convert .nc to geoTIFF
from: https://help.marine.copernicus.eu/en/articles/5029956-how-to-convert-netcdf-to-geotiff

In [6]:
nc_file = xr.open_dataset('./data/ml_hfi_subset.nc')
nc_file

<xarray.Dataset>
Dimensions:                        (lat: 1011, lon: 1011)
Coordinates:
  * lat                            (lat) float64 30.0 30.01 ... 39.98 39.99
  * lon                            (lon) float64 -110.0 -110.0 ... -100.0 -100.0
Data variables:
    __xarray_dataarray_variable__  (lat, lon) float64 ...

In [7]:
ml_hfi = nc_file['__xarray_dataarray_variable__']

In [8]:
ml_hfi = ml_hfi.rio.set_spatial_dims(x_dim='lon', y_dim='lat')
ml_hfi.rio.crs

In [9]:
# Define the CRS projection
ml_hfi.rio.write_crs("epsg:4326", inplace=True)

<xarray.DataArray '__xarray_dataarray_variable__' (lat: 1011, lon: 1011)>
[1022121 values with dtype=float64]
Coordinates:
  * lat          (lat) float64 30.0 30.01 30.02 30.03 ... 39.97 39.98 39.99
  * lon          (lon) float64 -110.0 -110.0 -110.0 ... -100.0 -100.0 -100.0
    spatial_ref  int64 0

In [10]:
ml_hfi.rio.to_raster(r"ml_hfi2019.tiff")

In [11]:
import rasterio
image_file = "./data/ml_hfi2019.tiff"

mlhfi_image = rasterio.open(image_file)

In [12]:
mlhfi_image

<open DatasetReader name='./data/ml_hfi2019.tiff' mode='r'>